In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time

c:\Users\aurel\Documents\Travail\Rust\Rust-Spike-Sorting\spikeinterface_rust\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

In [4]:
rec = si.load("../data/128_channels/recording")
rec.noise_levels = si.get_noise_levels(rec, random_slices_kwargs=dict(num_chunks_per_segment=20, chunk_size=1000))

noise_level (no parallelization): 100%|██████████| 20/20 [00:00<00:00, 223.31it/s]


In [5]:
n = 1

In [9]:
S_rust=0
S_numba=0

for _ in range(n):
    
    start = time.time()
    peaks = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs={"n_jobs": -1, "mp_context": "spawn", "pool_engine":"thread", "progress_bar": False, "chunk_duration":"100ms"})
    S_numba += time.time() - start
    start = time.time()
    peaks = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs={"n_jobs": -1, "mp_context": "spawn", "pool_engine":"thread", "progress_bar": False, "chunk_duration":"100ms"})
    S_rust += time.time() - start
print(f"Average time over {n} runs for locally_exclusive with rust engine: {S_rust/n:.2f} s")
print(f"Average time over {n} runs for locally_exclusive with numba engine: {S_numba/n:.2f} s")

Compute noise levels time: 0.000 s
Compute peaks on chunk time (numba): 0.006 s
Compute peaks on chunk time (numba): 0.006 s
Compute peaks on chunk time (numba): 0.008 s
Compute peaks on chunk time (numba): 0.006 s
Compute peaks on chunk time (numba): 0.007 s
Compute peaks on chunk time (numba): 0.006 s
Compute peaks on chunk time (numba): 0.007 s
Compute peaks on chunk time (numba): 0.007 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.003 s
Compute peaks on chunk time (numba): 0.003 s
Compute peaks on chunk time (numba): 0.003 s
Compute peaks on chunk time (numba): 0.006 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.005 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.005 s
Compute peaks on chunk time (numba): 0.004 s
Compute peaks on chunk time (numba): 0.005 s
Compute peaks on chu

In [ ]:
%timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs={"n_jobs": -1, "mp_context": "spawn", "pool_engine":"thread", "progress_bar": False, "chunk_duration":"1s"})

Compute noise levels time: 0.000 s


ValueError: If n_jobs>1 pool_engine must be 'process' or 'thread'